In [ ]:
import deepxde as dde
import numpy as np

import yaml
import pandas as pd
import time
import datetime
import os

#from deepxde.backend import tf
#from deepxde.backend import torch
import torch
from scipy.special import gamma
from scipy.special import lambertw

#import mayavi.mlab as mlab
import matplotlib.pyplot as plt

from draw_counter import random_sphere_points, uniform_sample_points
from PIL import Image

torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [ ]:
## 读取Config文件
with open('./Config/train_20220715.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    print(config)
config['NetWidth']

In [ ]:
## 单位统一至cm，定义各个参数
D = 0.585
Rsd = 17.5573
#Rwall = 2.654
Rwall = 2.56401
rpump = 0.75
q = 5
rcell = 1.5

Rrel = Rsd + q*Rwall
I = config['LightIntensity']*1e-3*1e4 # kg*(cm)^2/s^3
T = config['Temperature']
k1 = 10**(-20)*10**8 # kg*cm^4/s^2
k2 = 10**(-17)*10**4 # cm^2
k3 = 1 # cm^-3
sigm = 2.34601*10**(-13) # cm^2

n = k3*(1/(T+273.15))*10**(21.866+4.402-4453/(T+273.15))

In [ ]:
'''
读取Ref文件，COMSOL仿真结果
'''

# 读取z-axis实验结果
filename = './COMSOL_data/dual_light_zaxis.csv'
#df = pd.read_csv(filename, header=9, names=['x', 'y', 'z', 'Pz'])
df = pd.read_csv(filename, header=9, names=['Pz'])
ZAxis_COMSOL = np.asarray(df['Pz'])

# 读取3D仿真结果文件
filename_3D = './COMSOL_data/3D_dual_light.csv'
#df = pd.read_csv(filename, header=9, names=['x', 'y', 'z', 'Pz'])
df_3D = pd.read_csv(filename_3D, header=9, names=['x', 'y', 'z', 'Pz'])
axis_3D_x = np.asarray(df_3D['x'])
axis_3D_y = np.asarray(df_3D['y'])
axis_3D_z = np.asarray(df_3D['z'])
axis_3D_Pz = np.asarray(df_3D['Pz'])
axis_3D_axis = np.stack((axis_3D_x, axis_3D_y, axis_3D_z), axis=1)

## PINNs，计算z-axis的极化率数据. z_fitness 拟合结果

def Z_Axis(sample_num=300, rcell=1.5):
    x = 0
    y = 0
    z_sequence = np.linspace(-rcell, rcell, sample_num)
    sample_data = []
    
    for z in z_sequence:
        sample_data.append((x, y, z))
        
    sample_data = np.asarray(sample_data, dtype=np.float32)
    return sample_data


def fitness_metric(y_pred, y_ref):
    # 优化度拟合矩阵
    y_l2_error = np.linalg.norm(y_pred - y_ref, ord=2) # L2 范数
    y_pred_sum = np.sum(y_pred**2)
    
    fitness = 1 - np.sqrt(y_l2_error / y_pred_sum)
    return fitness

In [ ]:
def ode_system(x_axis, y_axis):
    ''' 输入z坐标，输出(Pz, R1, R2)'''
    Pz = y_axis[:, 0:1]
    R1 = y_axis[:, 1:2]
    R2 = y_axis[:, 2:3]
    
    Rop = R1 + R2
    dP_zz = dde.grad.hessian(y_axis, x_axis, i=0, j=0)
    eqn1 = -D*dP_zz + 1/q*(Rop+Rrel)*Pz - 1/q
    
    dR1 = dde.grad.jacobian(y_axis, x_axis, i=1, j=0) # 小心一点
    dR2 = dde.grad.jacobian(y_axis, x_axis, i=2, j=0)
    eqn2 = dR1 + n*R1*(1-Pz)
    eqn3 = dR2 - n*R2*(1-Pz)
    
    return [eqn1, eqn2, eqn3]

def func_boundary(x_axis, y_axis, numpy_X):
    Pz = y_axis[:, 0:1]
    R1 = y_axis[:, 1:2]
    R2 = y_axis[:, 2:3]
    
    Rop = R1 + R2
    dPz_z = dde.grad.jacobian(y_axis, x_axis, i=0, j=0)
    
    eqn = D*dPz_z + torch.sqrt(torch.abs(Rop*D/2))*Pz
    return eqn


#geom = dde.geometry.Sphere([0, 0, 0], rcell)
geom = dde.geometry.Interval(-1.5, 1.5)
bc = dde.icbc.OperatorBC(geom, func_boundary, lambda _, on_boundary: on_boundary)    
    
## 定义R1、R2的已知点
ob_pz = config['LightIntensity']
observe_points_R1 = dde.icbc.PointSetBC([0], ob_pz, component=1)
observe_points_R2 = dde.icbc.PointSetBC([0], ob_pz, component=2)

data = dde.data.PDE(geom, ode_systemtemtem,\
                    [bc, observe_points_R1, observe_points_R2],\
                    num_domain=500, num_boundary=50,\
                   anchors=observe_axis)

net = dde.nn.FNN([3] + [60] * 4 + [3], "tanh", "Glorot normal")
model = dde.Model(data, net)
#model.compile("adam", lr=1e-4, external_trainable_variables=[q_lambda])
model.compile("adam", lr=1e-4)
losshistory, train_state = model.train(epochs=3000)
    